In [1]:
%load_ext autoreload
%autoreload 2

# Добавляем путь к основной папке проекта, чтобы иметь возможность делать импорт из src
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pickle
import tqdm

import pandas as pd

from censorability_monitor.mempool_analysis import get_transaction_status, divide_mempool_to_three_parts

In [3]:
tr_data = pd.read_csv('../data/merged_221220_221231_with_contracts.csv')
tr_data.head()

,tx_hash,from,to,gas,gasPrice,blockNumber,timestamp_x,miner,timestamp_y,waiting,...,is_contract_transaction,baseFeePerGas,wait_time_adjusted,is_ofac,priority_fee,maxFeePerGas,maxPriorityFeePerGas,gasUsed,effectiveGasPrice,nonce
0,0x14e970dba71ef93b42f337d3494c4d6e8fee4d83ce38...,0x498B859D2e59958E209d7DD262324C8D31b12B12,0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5,242494,117.114005,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,NaN,NaN,...,True,17.248596,0.0,False,99.865409,117.114005,117.114005,189564,117.114005,12269
1,0x33fcc824c05553a3ec31a7d71dc6dbb26cf51d608009...,0x07B873DE48807ec7B02B38cc31b4f96a6A507182,0x883B01205b938EC7268286d13Cb40a2045D2E78F,800000,23.932889,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,NaN,NaN,...,True,17.248596,0.0,False,6.684293,23.932889,23.932889,214272,23.932889,907
2,0x555ffb08812804e23fe18467886edaa167f502488c6b...,0xAC813c6b767F3C2f1764994F9D3E4D38A8df6F32,0x0b382C0A00D6080b0009F003C050eAC3c30f0000,305170,17.248596,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,NaN,NaN,...,True,17.248596,0.0,False,0.000000,17.248596,17.248596,152585,17.248596,1689
3,0x9febfdc63677cd0decb532cb049004add2959d7f1a1f...,0xf7C8133baEa3313993B3c476c016aa869C991679,0x91c8D7e4080bed28e26FCE1b87cACcFaF7bbF794,292432,18.212433,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,1.671484e+09,3.0,...,True,17.248596,3.0,False,0.963838,0.000000,0.000000,208202,18.212433,19
4,0xe24c2fb8e8d581d86710a0ca685e27ed6c36ca722d8e...,0xAC813c6b767F3C2f1764994F9D3E4D38A8df6F32,0x0b382C0A00D6080b0009F003C050eAC3c30f0000,245026,20.798797,16221291,1671483611,0x690B9A9E9aa1C9dB991C7721a92d351Db4FaC990,NaN,NaN,...,True,17.248596,0.0,False,3.550201,20.798797,20.798797,122513,20.798797,1690


In [4]:
tr_data.columns

Index(['tx_hash', 'from', 'to', 'gas', 'gasPrice', 'blockNumber',
       'timestamp_x', 'miner', 'timestamp_y', 'waiting', 'validator',
       'public_key', 'validator_name', 'validator_address',
       'pub_key_lido_status', 'validator_type', 'eth_datetime', 'eth_hour',
       'is_contract_transaction', 'baseFeePerGas', 'wait_time_adjusted',
       'is_ofac', 'priority_fee', 'maxFeePerGas', 'maxPriorityFeePerGas',
       'gasUsed', 'effectiveGasPrice', 'nonce'],
      dtype='object')

## Заполним для legacy транзакций поля maxFeePerGas и maxPriorityFeePerGas

In [5]:
print(len(tr_data[tr_data['maxFeePerGas'] == 0]))
print(len(tr_data[tr_data['maxPriorityFeePerGas'] == 0]))

2241866
2331638


In [6]:
tr_data.loc[tr_data['maxFeePerGas'] == 0, 'maxFeePerGas'] = tr_data.loc[tr_data['maxFeePerGas'] == 0, 'gasPrice']
tr_data.loc[tr_data['maxPriorityFeePerGas'] == 0, 'maxPriorityFeePerGas'] = tr_data.loc[tr_data['maxPriorityFeePerGas'] == 0, 'maxFeePerGas']

In [7]:
tr_data['first_seen'] = tr_data['timestamp_y']
tr_data.loc[tr_data['timestamp_y'].isna(), 'first_seen'] = tr_data.loc[tr_data['timestamp_y'].isna(), 'timestamp_x'] - 1

In [8]:
print(len(tr_data[tr_data['maxFeePerGas'] == 0]))
print(len(tr_data[tr_data['maxPriorityFeePerGas'] == 0]))

0
0


# Провайдеры данных о блокчейне

In [9]:
import web3
from web3 import Web3, HTTPProvider, IPCProvider 

geth_ipc = '/media/Warehouse/Warehouse/Ethereum/data/.ethereum/geth.ipc'
w3 = Web3(IPCProvider(geth_ipc))
w3_drpc = Web3(HTTPProvider('https://main.drpc.org/ogrpc?network=ethereum&dkey=AhVPNnFg1020t9FEr-MKVyPunnyCo98R7bAt5s-wSZOx'))
print ("Latest Ethereum block number DRPC" , w3_drpc.eth.blockNumber)
print ("Latest Ethereum block number" , w3.eth.blockNumber)

Latest Ethereum block number DRPC 16562993
Latest Ethereum block number 16562993


In [29]:
# tx_by_address = tr_data[['from', 'tx_hash', 'nonce', 'blockNumber']].groupby(
#     ['from', 'tx_hash']
# ).agg({'nonce': 'first', 'blockNumber': 'first'})

# Анализ того как распределены транзакции по времени ожидания в текущем и следующем блоках

In [16]:
first_block_n = tr_data['blockNumber'].min()
last_block_n = tr_data['blockNumber'].max()

In [17]:
fast_transactions = []

for b_number in tqdm.tqdm(range(first_block_n, last_block_n + 1)):
    current_block_tx, next_block_tx, after_next_block_tx =  divide_mempool_to_three_parts(
        b_number, tr_data, w3)

    block_fast_transactions = {}

    current_block_counts = current_block_tx['already_waiting'].astype(int).value_counts()
    next_block_counts = next_block_tx['already_waiting'].astype(int).value_counts()
    after_next_block_counts = after_next_block_tx['already_waiting'].astype(int).value_counts()
    for t in range(13):
        block_fast_transactions[f'current_{t}'] = 0
        if t in current_block_counts.index:
            block_fast_transactions[f'current_{t}'] = current_block_counts.loc[t]
        block_fast_transactions[f'next_{t}'] = 0
        if t in next_block_counts.index:
            block_fast_transactions[f'next_{t}'] = next_block_counts.loc[t]

    fast_transactions.append(block_fast_transactions)

 24%|█████████████████▋                                                       | 20822/86002 [55:57<2:55:10,  6.20it/s]


KeyboardInterrupt: 

In [18]:
len(fast_transactions)

20822

In [19]:
stats = pd.DataFrame.from_records(fast_transactions)
stats.index = list(range(first_block_n, first_block_n + len(fast_transactions)))

# shares = pd.DataFrame()
# for t in range(13):
#     shares[f'current_{t}'] = stats[f'current_{t}'] / (stats[f'current_{t}'] + stats[f'next_{t}'])

### Транзакции, время прохождения которых не более 24 секунд (то есть либо сразу попадают в блок после появления, либо в следующий за ним):

In [20]:
for t in range(1, 13):
    print(f"t = {t}: {stats[f'current_{t}'].sum() / (stats[f'current_{t}'].sum() + stats[f'next_{t}'].sum()) * 100:0.2f}%")

t = 1: 84.43%
t = 2: 87.64%
t = 3: 90.20%
t = 4: 92.69%
t = 5: 94.38%
t = 6: 94.74%
t = 7: 94.98%
t = 8: 95.24%
t = 9: 95.34%
t = 10: 96.88%
t = 11: 98.28%
t = 12: 98.40%


### Всего таких транзакций в нашем датасете:

In [21]:
n_fast = len(tr_data[tr_data['wait_time_adjusted'] <= 24])
print(f'Всего {n_fast} ({n_fast / len(tr_data) * 100:0.2f})')

Всего 10921387 (94.74)


# Анализ состояния транзакций не попавших в текущий блок

In [41]:
def get_addresses_history_data(b_number: int):
    with open(f'../block_addresses_details/block_{b_number}_address_details.pickle', 'rb') as f:
        tx_additional_details = pickle.load(f)

    cached_address_balances = {}
    cached_address_n_transactions = {}
    for addr, details in tx_additional_details.items():
        cached_address_balances[addr] = details['account_balance']
        cached_address_n_transactions[addr] = details['tx_count']
        
    return cached_address_balances, cached_address_n_transactions

# Определим "интересные блоки"

In [186]:
ofac_txs = tr_data[tr_data['is_ofac'] == True].copy()
# Соберем номера блоков, которые нам интересны (должны содержать в мемпуле ofac транзакции)
ofac_txs.head()
interesting_blocks = set()

for i, row in ofac_txs.iterrows():
    block_with_ofac = row['blockNumber']
    waiting_time = row['wait_time_adjusted']
    n_blocks_before = int(waiting_time) // 12 + 1
    new_interesting_blocks = list(range(block_with_ofac, block_with_ofac - n_blocks_before - 1, -1))
    interesting_blocks.update(new_interesting_blocks)

first_block = tr_data['blockNumber'].min()
first_20_blocks = list(range(first_block + 1, first_block + 21))

interesting_blocks = set(interesting_blocks)
interesting_blocks.update(first_20_blocks)
interesting_blocks = sorted(interesting_blocks)

In [314]:
# Не по всем блокам есть исторические данные

interesting_blocks_with_info = [k for k in interesting_blocks if k <= 16253259]
len(interesting_blocks_with_info)

2200

### Допущение - транзакции со временем ожидания <=24 секунды не цензурируются, а просто не успевают попасть в блок

In [237]:
blocks_suspicious_txs = {}

# b_number = 16221292 # last 16221311

# for b_number in range(16221292, 16221293):
for b_number in tqdm.tqdm(interesting_blocks_with_info):
    block_analysis = {}
    transaction_statuses = {}
    block_info = w3.eth.get_block(b_number)
    baseFeePerGas = block_info['baseFeePerGas'] / 10 ** 9

    current_block_tx, next_block_tx, after_next_block_tx =  divide_mempool_to_three_parts(
        b_number, tr_data, w3)

    # Кэшируем данные об аккаунтах (уже загрузили с drpc для части блоков)

    cached_address_balances, cached_address_n_transactions = get_addresses_history_data(b_number)

    grouped_txs = pd.concat(
        [current_block_tx, next_block_tx, after_next_block_tx]
    ).groupby(['from', 'tx_hash']).agg('first')
    for address in grouped_txs.index.get_level_values('from').unique():
        transactions_from_addr = grouped_txs.loc[address].sort_values('nonce', ascending=True)
        # address_transaction_number = w3_drpc.eth.get_transaction_count(address, block_number - 1)
        # account_balance = w3_drpc.eth.get_balance(address, block_number - 1) / 10 ** 18
        address_transaction_number = cached_address_n_transactions[address]
        account_balance = cached_address_balances[address]

        if len(transactions_from_addr) == 1:
            # 1 transaction from address
            tx_hash = transactions_from_addr.index[0]
            transaction = transactions_from_addr.iloc[0]

            transaction_statuses[tx_hash] = get_transaction_status(
                tx_hash, transaction, address_transaction_number, account_balance, b_number, baseFeePerGas, w3)
            # continue

        else:
            # Транзакций с адреса больше чем одна
            for i, (tx_hash, transaction) in enumerate(transactions_from_addr.iterrows()):
                status = get_transaction_status(
                    tx_hash, transaction, address_transaction_number + i, account_balance, b_number, baseFeePerGas, w3)
                transaction_statuses[tx_hash] = status

                if status != 'ok':
                    # Нужно проставить этот же статус всем остальным транзакциям с этого адреса
                    break
            for tx_hash in transactions_from_addr.index[i + 1:]:
                transaction_statuses[tx_hash] = f'{status}'

    current_block_tx['status'] = current_block_tx['tx_hash'].apply(lambda x: transaction_statuses[x])
    next_block_tx['status'] = next_block_tx['tx_hash'].apply(lambda x: transaction_statuses[x])
    after_next_block_tx['status'] = after_next_block_tx['tx_hash'].apply(lambda x: transaction_statuses[x])

    # Filter out not eligible transactions
    mempool_with_statuses = pd.concat([current_block_tx, next_block_tx, after_next_block_tx])
    mempool_with_statuses = mempool_with_statuses[mempool_with_statuses['status'].isin(['ok', 'low_eth'])].copy()
    mempool_with_statuses.sort_values('availablePriorityFee', ascending=False, inplace=True)
    n = len(mempool_with_statuses)
    mempool_with_statuses['priority_number'] = range(n)
    mempool_with_statuses['status'].value_counts()

    # Add time when included into block
    mempool_with_statuses['included'] = ''
    mempool_with_statuses.loc[mempool_with_statuses['blockNumber'] == b_number, 'included'] = 'now'
    mempool_with_statuses.loc[mempool_with_statuses['blockNumber'] == b_number + 1, 'included'] = 'next_block'
    mempool_with_statuses.loc[mempool_with_statuses['blockNumber'] > b_number + 1, 'included'] = 'in_future'

    total_gas_in_txs = mempool_with_statuses['gasUsed'].sum()
    
    block_analysis['n_elig_tx'] =  len(current_block_tx) + len(next_block_tx) + len(after_next_block_tx)
    block_analysis['n_elig_tx_current_block'] =  len(current_block_tx)
    block_analysis['n_elig_tx_next_block'] =  len(next_block_tx)
    block_analysis['n_elig_tx_future'] =  len(after_next_block_tx)
    block_analysis['eligible_tx_gas'] = total_gas_in_txs
    block_analysis['n_elig_0_4_cur_block'] = len(current_block_tx[current_block_tx['already_waiting'] < 5])
    block_analysis['n_elig_5_8_cur_block'] = len(current_block_tx[current_block_tx['already_waiting'].between(5, 8)])
    block_analysis['n_elig_9_12_cur_block'] = len(current_block_tx[current_block_tx['already_waiting'].between(9, 12)])
    
    fast_tx_next_block = mempool_with_statuses[(mempool_with_statuses['already_waiting'] < 13)
                                               & (mempool_with_statuses['included'] == 'next_block')].copy()
    
    other_than_fast_next_block_tx = mempool_with_statuses[
        ~mempool_with_statuses['tx_hash'].isin(fast_tx_next_block['tx_hash'])
    ].copy()
    
    # Сюда включаем все транзакции кроме тех, которые попадают в след блок (с общим временем ожидания до 24 сек)
    # Идея такая - их не подозреваем, потому что они скорее всего не цензурируются, а просто не успели попасть в блок
    can_be_included = other_than_fast_next_block_tx[other_than_fast_next_block_tx['status'].isin(['ok', 'low_eth'])]
    can_be_included['cum_gas_sum'] = can_be_included['gasUsed'].cumsum()
    can_be_included = can_be_included[can_be_included['cum_gas_sum'] <= 30000000]
    
    suspicious_df = can_be_included[can_be_included['included'] != 'now']
    
    suspicious_txs = suspicious_df[['tx_hash', 'is_ofac', 'already_waiting', 'status', 'validator_type']]
    
    block_analysis['n_suspicious'] = len(suspicious_df)
    block_analysis['n_suspicious_ok'] = len(suspicious_df[suspicious_df['status'] == 'ok'])
    block_analysis['n_suspicious_low_eth'] = len(suspicious_df[suspicious_df['status'] == 'low_eth'])
    block_analysis['n_suspicious_ofac'] = suspicious_df['is_ofac'].sum()
    block_analysis['n_ofac'] = mempool_with_statuses['is_ofac'].sum()
    block_analysis['n_ofac_current_block'] = mempool_with_statuses[mempool_with_statuses['included'] == 'now']['is_ofac'].sum()
    block_analysis['n_ofac_next_block'] = mempool_with_statuses[mempool_with_statuses['included'] == 'next_block']['is_ofac'].sum()
    block_analysis['n_ofac_future'] = mempool_with_statuses[mempool_with_statuses['included'] == 'in_future']['is_ofac'].sum()
    block_analysis['n_ofac_0_4_cur_block'] = mempool_with_statuses[(mempool_with_statuses['included'] == 'now')
                                                                   & (mempool_with_statuses['already_waiting'] < 5)]['is_ofac'].sum()
    block_analysis['n_ofac_5_8_cur_block'] = mempool_with_statuses[(mempool_with_statuses['included'] == 'now')
                                                                   & (mempool_with_statuses['already_waiting']).between(5, 8)]['is_ofac'].sum()
    block_analysis['n_ofac_9_12_cur_block'] = mempool_with_statuses[(mempool_with_statuses['included'] == 'now')
                                                                    & (mempool_with_statuses['already_waiting'].between(9, 12))]['is_ofac'].sum()
    block_analysis['suspicious'] = suspicious_txs
    blocks_suspicious_txs[b_number] = block_analysis

100%|███████████████████████████████████████████████████████████████████████████████| 905/905 [13:16<00:00,  1.14it/s]


In [238]:
interesting_blocks_stats = []
for k, v in blocks_suspicious_txs.items():
    item = v.copy()
    del item['suspicious']
    item['blockNumber'] = k
    interesting_blocks_stats.append(item)

df = pd.DataFrame.from_dict(interesting_blocks_stats).sort_values('blockNumber')

In [239]:
df[df['n_ofac'] > 0].head(50)

,n_elig_tx,n_elig_tx_current_block,n_elig_tx_next_block,n_elig_tx_future,eligible_tx_gas,n_elig_0_4_cur_block,n_elig_5_8_cur_block,n_elig_9_12_cur_block,n_suspicious,n_suspicious_ok,n_suspicious_low_eth,n_suspicious_ofac,n_ofac,n_ofac_current_block,n_ofac_next_block,n_ofac_future,n_ofac_0_4_cur_block,n_ofac_5_8_cur_block,n_ofac_9_12_cur_block,blockNumber
21,702,171,0,531,16858610,63,77,31,16,5,11,0,1,1,0,0,1,0,0,16221337
23,533,148,0,385,20922500,55,71,22,8,5,3,0,1,1,0,0,0,0,1,16221817
25,501,111,1,389,19418749,46,37,25,8,5,3,0,1,0,1,0,0,0,0,16221823
26,482,103,21,358,13027743,26,40,36,8,6,2,0,2,2,0,0,0,0,1,16221824
28,806,119,0,687,16690053,28,47,34,26,13,13,1,1,0,0,1,0,0,0,16222094
29,803,115,3,685,15171695,42,43,30,26,13,13,1,1,0,1,0,0,0,0,16222095
30,834,120,4,710,49964677,43,38,32,27,14,13,0,1,1,0,0,0,0,0,16222096
34,856,121,0,735,14838800,36,31,32,24,11,13,0,1,1,0,0,1,0,0,16222762
36,966,114,13,839,639366176,24,24,55,23,12,11,0,1,1,0,0,0,1,0,16222765
38,812,100,13,699,22526086,34,20,36,32,16,16,0,1,1,0,0,0,1,0,16223156


# Вариант из тетрадки

In [315]:
blocks_suspicious_txs = {}
mempools_for_classification = []

# b_number = 16221292 # last 16221311

# for b_number in range(16221292, 16221293):
for b_number in tqdm.tqdm(interesting_blocks_with_info):

    # b_number = 16221292
    block_analysis = {}
    transaction_statuses = {}
    block_info = w3.eth.get_block(b_number)
    baseFeePerGas = block_info['baseFeePerGas'] / 10 ** 9

    current_block_tx, next_block_tx, after_next_block_tx =  divide_mempool_to_three_parts(
        b_number, tr_data, w3)

    # Кэшируем данные об аккаунтах (уже загрузили с drpc для части блоков)

    cached_address_balances, cached_address_n_transactions = get_addresses_history_data(b_number)

    grouped_txs = pd.concat(
        [current_block_tx, next_block_tx, after_next_block_tx]
    ).groupby(['from', 'tx_hash']).agg('first')
    for address in grouped_txs.index.get_level_values('from').unique():
        transactions_from_addr = grouped_txs.loc[address].sort_values('nonce', ascending=True)
        # address_transaction_number = w3_drpc.eth.get_transaction_count(address, block_number - 1)
        # account_balance = w3_drpc.eth.get_balance(address, block_number - 1) / 10 ** 18
        address_transaction_number = cached_address_n_transactions[address]
        account_balance = cached_address_balances[address]

        if len(transactions_from_addr) == 1:
            # 1 transaction from address
            tx_hash = transactions_from_addr.index[0]
            transaction = transactions_from_addr.iloc[0]

            transaction_statuses[tx_hash] = get_transaction_status(
                tx_hash, transaction, address_transaction_number, account_balance, b_number, baseFeePerGas, w3)
            # continue

        else:
            # Транзакций с адреса больше чем одна
            for i, (tx_hash, transaction) in enumerate(transactions_from_addr.iterrows()):
                status = get_transaction_status(
                    tx_hash, transaction, address_transaction_number + i, account_balance, b_number, baseFeePerGas, w3)
                transaction_statuses[tx_hash] = status

                if status != 'ok':
                    # Нужно проставить этот же статус всем остальным транзакциям с этого адреса
                    break
            for tx_hash in transactions_from_addr.index[i + 1:]:
                transaction_statuses[tx_hash] = f'{status}'

    current_block_tx['status'] = current_block_tx['tx_hash'].apply(lambda x: transaction_statuses[x])
    next_block_tx['status'] = next_block_tx['tx_hash'].apply(lambda x: transaction_statuses[x])
    after_next_block_tx['status'] = after_next_block_tx['tx_hash'].apply(lambda x: transaction_statuses[x])

    # Filter out not eligible transactions
    mempool_with_statuses = pd.concat([current_block_tx, next_block_tx, after_next_block_tx])
    mempool_with_statuses = mempool_with_statuses[mempool_with_statuses['status'].isin(['ok', 'low_eth'])].copy()

    # print(mempool_with_statuses.columns)

    mempool_with_statuses.sort_values('availablePriorityFee', ascending=False, inplace=True)
    n = len(mempool_with_statuses)
    mempool_with_statuses['priority_number'] = range(n)
    mempool_with_statuses['status'].value_counts()

    # Add time when included into block
    mempool_with_statuses['included'] = ''
    mempool_with_statuses.loc[mempool_with_statuses['blockNumber'] == b_number, 'included'] = 'now'
    mempool_with_statuses.loc[mempool_with_statuses['blockNumber'] == b_number + 1, 'included'] = 'next_block'
    mempool_with_statuses.loc[mempool_with_statuses['blockNumber'] > b_number + 1, 'included'] = 'in_future'

    mempool_with_statuses['current_block'] = b_number
    mempool_with_statuses['timestamp'] = block_info['timestamp']
    mempool_with_statuses['baseFeePerGas'] = baseFeePerGas
    prev_block_info = w3.eth.get_block(b_number - 1)
    mempool_with_statuses['previousBlockGasUsed'] = prev_block_info['gasUsed']

    mempool_current_data = mempool_with_statuses[
        ['tx_hash', 'current_block', 'timestamp', 'status',
         'availablePriorityFee', 'gas', 'already_waiting',
         'baseFeePerGas', 'previousBlockGasUsed', 'is_ofac', 'included']].copy()

    mempool_current_data['priority_n'] = list(range(len(mempool_current_data)))
    mempool_current_data['total_eligible_txs'] = len(mempool_current_data)
    mempool_current_data['priority_percent'] = mempool_current_data['priority_n'] / mempool_current_data['total_eligible_txs']
    mempool_current_data.sort_values('availablePriorityFee', ascending=False, inplace=True)
    mempool_current_data['cumulative_gas'] = mempool_current_data['gas'].cumsum()
    mempool_current_data['first_gas_unit'] = mempool_current_data['cumulative_gas'].shift(1).fillna(0) / 30_000_000
    mempool_current_data['last_gas_unit'] = mempool_current_data['cumulative_gas'] / 30_000_000

    mempools_for_classification.append(mempool_current_data)

100%|█████████████████████████████████████████████████████████████████████████████| 2200/2200 [42:58<00:00,  1.17s/it]


In [316]:
classification_dataset = pd.concat(mempools_for_classification)

In [317]:
classification_dataset.to_csv('../data/classification_dataset.csv', index=False)

In [318]:
classification_dataset.head()

,tx_hash,current_block,timestamp,status,availablePriorityFee,gas,already_waiting,baseFeePerGas,previousBlockGasUsed,is_ofac,included,priority_n,total_eligible_txs,priority_percent,cumulative_gas,first_gas_unit,last_gas_unit
546,0x36a8f87e6be412d5d3ef5c774e221e610b5c8871463b...,16221292,1671483623,ok,1092.697098,175920,1.0,18.302902,22334900,False,next_block,0,346,0.000000,175920,0.000000,0.005864
396,0x1be4691644fd620612a44c306341d3e4e5c4486ada4e...,16221292,1671483623,ok,686.697098,175920,4.0,18.302902,22334900,False,now,1,346,0.002890,351840,0.005864,0.011728
547,0xe67a86403cbe1dd893b764c49bdc86d821fcbd81d996...,16221292,1671483623,ok,599.000000,175920,2.0,18.302902,22334900,False,next_block,2,346,0.005780,527760,0.011728,0.017592
397,0x98afadc301f16fb299311247444e77a422dbf35deddf...,16221292,1671483623,ok,511.000000,175920,5.0,18.302902,22334900,False,now,3,346,0.008671,703680,0.017592,0.023456
548,0x9ec221ab70707257d9343329a607667624423144b046...,16221292,1671483623,ok,444.000000,175920,2.0,18.302902,22334900,False,next_block,4,346,0.011561,879600,0.023456,0.029320


In [309]:
mempool_current_data['priority_n'] = list(range(len(mempool_current_data)))
mempool_current_data['total_eligible_txs'] = len(mempool_current_data)
mempool_current_data['priority_percent'] = mempool_current_data['priority_n'] / mempool_current_data['total_eligible_txs']
mempool_current_data.sort_values('availablePriorityFee', ascending=False, inplace=True)
mempool_current_data['cumulative_gas'] = mempool_current_data['gas'].cumsum()
mempool_current_data['first_gas_unit'] = mempool_current_data['cumulative_gas'].shift(1).fillna(0) / 30_000_000
mempool_current_data['last_gas_unit'] = mempool_current_data['cumulative_gas'] / 30_000_000
mempool_current_data

,tx_hash,current_block,timestamp,status,availablePriorityFee,gas,already_waiting,baseFeePerGas,previousBlockGasUsed,is_ofac,included,priority_n,total_eligible_txs,cumulative_gas,first_gas_unit,last_gas_unit,priority_percent
546,0x36a8f87e6be412d5d3ef5c774e221e610b5c8871463b...,16221292,1671483623,ok,1092.697098,175920,1.0,18.302902,22334900,False,next_block,0,346,175920,0.000000,0.005864,0.000000
396,0x1be4691644fd620612a44c306341d3e4e5c4486ada4e...,16221292,1671483623,ok,686.697098,175920,4.0,18.302902,22334900,False,now,1,346,351840,0.005864,0.011728,0.002890
547,0xe67a86403cbe1dd893b764c49bdc86d821fcbd81d996...,16221292,1671483623,ok,599.000000,175920,2.0,18.302902,22334900,False,next_block,2,346,527760,0.011728,0.017592,0.005780
397,0x98afadc301f16fb299311247444e77a422dbf35deddf...,16221292,1671483623,ok,511.000000,175920,5.0,18.302902,22334900,False,now,3,346,703680,0.017592,0.023456,0.008671
548,0x9ec221ab70707257d9343329a607667624423144b046...,16221292,1671483623,ok,444.000000,175920,2.0,18.302902,22334900,False,next_block,4,346,879600,0.023456,0.029320,0.011561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,0x7fa7ab9290c17e32fe1ed27f0c9307f58705c2d2843d...,16221292,1671483623,ok,0.263451,21000,9.0,18.302902,22334900,False,now,341,346,46275790,1.541826,1.542526,0.985549
543,0xb72ed02171aa52ad8c382ce59fdb1f4887465ff12134...,16221292,1671483623,ok,0.250000,90000,10.0,18.302902,22334900,False,now,342,346,46365790,1.542526,1.545526,0.988439
542,0x87626053310794144c6abc3bc24bb56cb2964ed21632...,16221292,1671483623,ok,0.250000,21000,15.0,18.302902,22334900,False,now,343,346,46386790,1.545526,1.546226,0.991329
541,0x586cd0e258cf12c480108cf3d6ced9fc1aa2f5132bb5...,16221292,1671483623,ok,0.250000,485318,15.0,18.302902,22334900,False,now,344,346,46872108,1.546226,1.562404,0.994220


In [299]:
mempool_current_data['cumulative_gas'].shift(1).fillna(0)

546           0.0
396      175920.0
547      351840.0
397      527760.0
548      703680.0
          ...    
539    46254790.0
543    46275790.0
542    46365790.0
541    46386790.0
540    46872108.0
Name: cumulative_gas, Length: 346, dtype: float64

# Что-то солучилось в этом блоке очень много адресов в мемпуле - 16253260